
In the beginning of this class, you identified emails by their authors using a number of supervised classification algorithms. In those projects, we handled the preprocessing for you, transforming the input emails into a TfIdf so they could be fed into the algorithms. Now you will construct your own version of that preprocessing step, so that you are going directly from raw data to processed features.

You will be given two text files: one contains the locations of all the emails from Sara, the other has emails from Chris. You will also have access to the `parseOutText()` function, which accepts an opened email as an argument and returns a string containing all the (stemmed) words in the email.

You’ll start with a warmup exercise to get acquainted with `parseOutText()`. Go to the *tools* directory and run *parse_out_email_text.py*, which contains `parseOutText()` and a test email to run this function over.

`parseOutText()` takes the opened email and returns only the text part, stripping away any metadata that may occur at the beginning of the email, so what's left is the text of the message. We currently have this script set up so that it will print the text of the email to the screen, **what is the text that you get when you run parseOutText()?**

`Answer: Hi Everyone  If you can read this message youre properly using parseOutText  Please proceed to the next part of the project`

==================================================================================================================

In `parseOutText()`, comment out the following line: 

`words = text_string` 

Augment `parseOutText()` so that the string it returns has all the words stemmed using a SnowballStemmer (use the nltk package, some examples that I found helpful can be found here: http://www.nltk.org/howto/stem.html ). **Rerun `parse_test.py`, which will use your updated `parseOutText()` function--what’s your output now?**

Hint: you'll need to break the string down into individual words, stem each word, then recombine all the words into one string.

In [10]:
#!/usr/bin/python

from nltk.stem.snowball import SnowballStemmer
#from sklearn.feature_extraction.text import CountVectorizer
import string

def parseOutText(f):
    """ given an opened email file f, parse out all text below the
        metadata block at the top
        (in Part 2, you will also add stemming capabilities)
        and return a string that contains all the words
        in the email (space-separated) 
        
        example use case:
        f = open("email_file_name.txt", "r")
        text = parseOutText(f)
        
        """


    f.seek(0)  ### go back to beginning of file (annoying)
    all_text = f.read()

    ### split off metadata
    content = all_text.split("X-FileName:")
    words = ""
    if len(content) > 1:
        ### remove punctuation
        text_string = content[1].translate(string.maketrans("", ""), string.punctuation)

        ### project part 2: comment out the line below
        ### words = text_string

        ### split the text string into individual words, stem each word,
        ### and append the stemmed word to words (make sure there's a single
        ### space between each stemmed word)
        text_lst = text_string.split()
        words = ""
        stemmer = SnowballStemmer("english")
        for each in text_lst:
            root = stemmer.stem(each)
            words += root + ' '

    
    return words

    

def main():
    ff = open("../text_learning/test_email.txt", "r")
    text = parseOutText(ff)
    print text



if __name__ == '__main__':
    main()

        

hi everyon if you can read this messag your proper use parseouttext pleas proceed to the next part of the project 


=================================================================================================================

In _vectorize_text.py_, you will iterate through all the emails from Chris and from Sara. For each email, feed the opened email to `parseOutText()` and return the stemmed text string. Then do two things:

remove signature words (“sara”, “shackleton”, “chris”, “germani”--bonus points if you can figure out why it's "germani" and not "germany")
append the updated text string to word_data -- if the email is from Sara, append 0 (zero) to from_data, or append a 1 if Chris wrote the email.
Once this step is complete, you should have two lists: one contains the stemmed text of each email, and the second should contain the labels that encode (via a 0 or 1) who the author of that email is.

Running over all the emails can take a little while (5 minutes or more), so we've added a temp_counter to cut things off after the first 200 emails. Of course, once everything is working, you'd want to run over the full dataset.

**In the box below, put the string that you get for word_data[152].**


In [4]:
#!/usr/bin/python

import os
import pickle
import re
import sys

sys.path.append( "../tools/" )
from parse_out_email_text import parseOutText

"""
    Starter code to process the emails from Sara and Chris to extract
    the features and get the documents ready for classification.

    The list of all the emails from Sara are in the from_sara list
    likewise for emails from Chris (from_chris)

    The actual documents are in the Enron email dataset, which
    you downloaded/unpacked in Part 0 of the first mini-project. If you have
    not obtained the Enron email corpus, run startup.py in the tools folder.

    The data is stored in lists and packed away in pickle files at the end.
"""


from_sara  = open("from_sara.txt", "r")
from_chris = open("from_chris.txt", "r")

from_data = []
word_data = []

### temp_counter is a way to speed up the development--there are
### thousands of emails from Sara and Chris, so running over all of them
### can take a long time
### temp_counter helps you only look at the first 200 emails in the list so you
### can iterate your modifications quicker
temp_counter = 0


for name, from_person in [("sara", from_sara), ("chris", from_chris)]:
    for path in from_person:
        ### only look at first 200 emails when developing
        ### once everything is working, remove this line to run over full dataset
        #temp_counter += 1
        if temp_counter < 200:
            path = os.path.join('..', path[:-1])
            #print path
            email = open(path, "r")
            ### use parseOutText to extract the text from the opened email
            mail_parsed = parseOutText(email)
            ### use str.replace() to remove any instances of the words
            ### ["sara", "shackleton", "chris", "germani"]
            mail_parsed.replace('sara',"")
            mail_parsed.replace('shackleton',"")
            mail_parsed.replace('chris',"")
            mail_parsed.replace('germani',"")
            mail_parsed.replace("  "," ")           
            ### append the text to word_data
            word_data.append(mail_parsed)
            ### append a 0 to from_data if email is from Sara, and 1 if email is from Chris
            if name == "sara":
                from_data.append(0)
            else:
                from_data.append(1)
            email.close()
#print word_data            
#print from_data
print len(from_data), "emails processed"
from_sara.close()
from_chris.close()

pickle.dump( word_data, open("your_word_data.pkl", "w") )
pickle.dump( from_data, open("your_email_authors.pkl", "w") )





### in Part 4, do TfIdf vectorization here



17578 emails processed


In [12]:
word_data[152]

u'tjonesnsf stephani and sam need nymex calendar '

=========================================================================================================

Transform the word_data into a tf-idf matrix using the sklearn TfIdf transformation. Remove english stopwords.

You can access the mapping between words and feature numbers using `get_feature_names()`, which returns a list of all the words in the vocabulary. **How many different words are there?**


In [18]:
#!/usr/bin/python

import os
import pickle
import re
import sys

sys.path.append( "../tools/" )
from parse_out_email_text import parseOutText

"""
    Starter code to process the emails from Sara and Chris to extract
    the features and get the documents ready for classification.

    The list of all the emails from Sara are in the from_sara list
    likewise for emails from Chris (from_chris)

    The actual documents are in the Enron email dataset, which
    you downloaded/unpacked in Part 0 of the first mini-project. If you have
    not obtained the Enron email corpus, run startup.py in the tools folder.

    The data is stored in lists and packed away in pickle files at the end.
"""


from_sara  = open("from_sara.txt", "r")
from_chris = open("from_chris.txt", "r")

from_data = []
word_data = []

### temp_counter is a way to speed up the development--there are
### thousands of emails from Sara and Chris, so running over all of them
### can take a long time
### temp_counter helps you only look at the first 200 emails in the list so you
### can iterate your modifications quicker
temp_counter = 0


for name, from_person in [("sara", from_sara), ("chris", from_chris)]:
    for path in from_person:
        ### only look at first 200 emails when developing
        ### once everything is working, remove this line to run over full dataset
        #temp_counter += 1
        if temp_counter < 200:
            path = os.path.join('..', path[:-1])
            #print path
            email = open(path, "r")
            ### use parseOutText to extract the text from the opened email
            mail_parsed = parseOutText(email)
            ### use str.replace() to remove any instances of the words
            ### ["sara", "shackleton", "chris", "germani"]
            mail_parsed.replace('sara',"")
            mail_parsed.replace('shackleton',"")
            mail_parsed.replace('chris',"")
            mail_parsed.replace('germani',"")
            mail_parsed.replace("  "," ")           
            ### append the text to word_data
            word_data.append(mail_parsed)
            ### append a 0 to from_data if email is from Sara, and 1 if email is from Chris
            if name == "sara":
                from_data.append(0)
            else:
                from_data.append(1)
            email.close()
#print word_data            
#print from_data
print len(from_data), "emails processed"
from_sara.close()
from_chris.close()

pickle.dump( word_data, open("your_word_data.pkl", "w") )
pickle.dump( from_data, open("your_email_authors.pkl", "w") )


#print word_data,"word_data length"
### in Part 4, do TfIdf vectorization here
from nltk.corpus import stopwords
sw = stopwords.words("english")

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(word_data)
feature_names = tfidf.get_feature_names()
print len(feature_names), "feature_names length"






17578 emails processed
44990 feature_names length


=============================================================================================


**What is word number 34597 in your TfIdf?**

(Just to be clear--if the question were "what is word number 100," we would be looking for the word corresponding to vocab_list[100]. Zero-indexed arrays are so confusing to talk about sometimes.)


In [19]:
feature_names[34597]

u'pleading'